In [11]:
import tensorflow as tf
import os

**load the data**

In [12]:
currentDir = os.getcwd()

filePath = os.path.join(currentDir, "Flower Classification Dataset/train")

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    filePath,
    labels = 'inferred',
    image_size = (224,224),
    batch_size = 102,
    validation_split = 0
)

Found 4102 files belonging to 102 classes.


**Outline the model**

In [13]:
input_size = 50176
output_size = 102
hidden_layer_size = 128 #TODO: figure out the exact value

model = tf.keras.Sequential([
    # ReScale pixel values in the band [0, 1] 
    tf.keras.Input(shape=(224, 224, 3)),
    tf.keras.layers.Rescaling(1./255),

    # Augmentation (Adding randomization to the images)
    # ONLY USED IN TRAINING
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.2),
    
    # Conv 1
    tf.keras.layers.Conv2D(
        32, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001),
        padding='same'
    ), #TODO: Check if the combination of L1 and L2 works better for all layers
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    # Conv 2
    tf.keras.layers.Conv2D(
        64, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001),
        padding='same'
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    # Conv 3
    tf.keras.layers.Conv2D(
        64, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001),
        padding='same'
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    # Conv 4
    tf.keras.layers.Conv2D(
        128, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001),
        padding='same'
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    # Flattened Inputs
    tf.keras.layers.GlobalAveragePooling2D(), # Better than normal Flattening, reduces the dimensionality of feature maps
        #tf.keras.layers.Flatten(),
    
    # Dense Layers (Fully connected layers (FC Layers))
    tf.keras.layers.Dense(
        hidden_layer_size, 
        activation = 'relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    #TODO: Check if dropping neurons actually prevents overfitting 
    #TODO: We can test other techniques besides dropping 
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(        
        hidden_layer_size, 
        activation = 'relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    tf.keras.layers.Dropout(0.3),

    # Didn't specify "softmax" activation function in the last layer 
    # to use the right order of opertions for loss calculations
    tf.keras.layers.Dense(output_size) 
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip_1 (RandomFlip)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_1               │ (None, 224, 224, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_1 (RandomZoom)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_contrast_1               │ (None, 224, 224, 3)    │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 128)            │             

 Total params: 177,510 (693.40 KB)

 Trainable params: 176,934 (691.15 KB)

 Non-trainable params: 576 (2.25 KB)

In [14]:
lossFunction = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) # Internally applies "softmax" if "from_logits = True"

model.compile(loss=lossFunction)